## Testing how weighted class works in Keras

In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam, SGD
import matplotlib.pylab as plt
%matplotlib inline 
from numpy import linalg as LA

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using Theano backend.
Using gpu device 2: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 4007)


## functions

In [2]:
def larger_model(params):
    lr=params['lr']
    # create model
    model = Sequential()
    model.add(Convolution2D(30, 5, 5, border_mode='valid', input_shape=(1, 28, 28),
    activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(15, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
    return model

## loading data

In [3]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

print 'data shape', X_train.shape,y_train.shape
print 'data shape', X_test.shape,y_test.shape

data shape (60000, 1, 28, 28) (60000, 10)
data shape (10000, 1, 28, 28) (10000, 10)


In [4]:
print 'wait to train ...'

params={'lr': 1e-3}
# build the model
model = larger_model(params)
model.summary()

wait to train ...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 24, 24)    780         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 30, 12, 12)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 15, 10, 10)    4065        maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 15, 5, 5)      0           convolution2d_2[0][0]            
_________________________________________________________________________

In [5]:
# Fit the model
cw={0:1, 1:1, 2:1,3:1,4:1,5:0,6:1,7:1,8:1,9:1}
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=200,verbose=1,class_weight=cw)


# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s - loss: 0.3509 - acc: 0.8064 - val_loss: 1.3484 - val_acc: 0.8882
Epoch 2/10
60000/60000 [==============================] - 1s - loss: 0.0859 - acc: 0.8852 - val_loss: 1.2937 - val_acc: 0.8964
Epoch 3/10
60000/60000 [==============================] - 1s - loss: 0.0611 - acc: 0.8916 - val_loss: 1.3620 - val_acc: 0.9000
Epoch 4/10
60000/60000 [==============================] - 1s - loss: 0.0497 - acc: 0.8951 - val_loss: 1.3462 - val_acc: 0.9012
Epoch 5/10
60000/60000 [==============================] - 1s - loss: 0.0430 - acc: 0.8976 - val_loss: 1.3750 - val_acc: 0.9012
Epoch 6/10
60000/60000 [==============================] - 1s - loss: 0.0379 - acc: 0.8989 - val_loss: 1.3300 - val_acc: 0.9029
Epoch 7/10
60000/60000 [==============================] - 1s - loss: 0.0341 - acc: 0.8998 - val_loss: 1.3742 - val_acc: 0.9043
Epoch 8/10
60000/60000 [==============================] - 1s 

In [6]:
# evaluate
score,accuracy=model.evaluate(X_test,y_test)
print '\n accuracy: %.2f' %accuracy

 9984/10000 [============================>.] - ETA: 0s
 accuracy: 0.90
